In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Chronic_Fatigue_Syndrome/GSE39684'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Comprehensive Investigation of Archival and Prospectively Collected Samples Reveals No Association of the XMRV Gammaretrovirus with Prostate Cancer"
!Series_summary	"XMRV, or xenotropic murine leukemia virus (MLV)-related virus, is a novel gammaretrovirus originally identified in studies that analyzed tissue from prostate cancer patients in 2006 and blood from patients with chronic fatigue syndrome (CFS) in 2009. However, a large number of subsequent studies failed to confirm a link between XMRV infection and CFS or prostate cancer. On the contrary, recent evidence indicates that XMRV is a contaminant originating from the recombination of two mouse endogenous retroviruses during passaging of a prostate tumor xenograft (CWR22) in mice, generating laboratory-derived cell lines that are XMRV-infected. To confirm or refute an association between XMRV and prostate cancer, we analyzed prostate cancer tissues and plasma from a prospectively collected coh

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable


### Step 2: Dataset Analysis and Questions: Unknown Step

In [4]:

# 1. Gene Expression Data Availability
is_gene_available = True  # Since the dataset uses ViroChip microarray targeting RNA expression.

# 2. Variable Availability and Data Type Conversion
sample_characteristics_dict = {
    0: ['cell line: 22Rv1', 'tissue: prostate biopsy'], 
    1: ['polya enrichment: yes', 'polya enrichment: no', 'cohort: 2006', 'cohort: 2012'], 
    2: [float('nan'), 'polya enrichment: yes', 'polya enrichment: no']
}

# 2.1 Data Availability
trait_row = None  # No direct information on 'Chronic_Fatigue_Syndrome'.
age_row = None  # No direct information on 'age'.
gender_row = None  # No direct information on 'gender'.

# Functions to convert data values to the correct type
# Continuous: Age (Numerical)
def convert_age(value):
    try:
        return float(value.split(':')[1].strip()) if ':' in value else None
    except ValueError:
        return None

# Binary: Trait (Chronic Fatigue Syndrome) and Gender
# Gender Conversion
def convert_gender(value):
    if ':' in value:
        val = value.split(':')[1].strip().lower()
        if val == 'female':
            return 0
        elif val == 'male':
            return 1
    return None

# Trait Conversion (Placeholder, as trait_row is None)
def convert_trait(value):
    return None

# 3. Save Metadata
save_cohort_info('GSE39684', './preprocessed/Chronic_Fatigue_Syndrome/cohort_info.json', is_gene_available, trait_row is not None)

# 4. Clinical Feature Extraction
# Since trait_row is None, clinical feature extraction and saving is skipped.
